In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None
from io import StringIO
import numpy as np
import math

In [5]:
G06 = StringIO('''
10 45 0.00000000 ,577.445165
10 30 0.00000000 ,577.441900
10 15 0.00000000 ,577.438634
10 0 0.00000000 , 577.435389
9 45 0.00000000 , 577.432105
9 30 0.00000000 , 577.428825
9 15 0.00000000 , 577.425556
9 0 0.00000000 ,  577.422289
8 45 0.00000000 , 577.419003
8 30 0.00000000 , 577.415707''')

In [6]:
G11 = StringIO('''
10 45 0.00000000 ,-117.548696
10 30 0.00000000 ,-117.538965
10 15 0.00000000 ,-117.529230
10 0 0.00000000 , -117.519503
9 45 0.00000000 , -117.509796
9 30 0.00000000 , -117.500047
9 15 0.00000000 , -117.490327
9 0 0.00000000 ,  -117.480625
8 45 0.00000000 , -117.470932
8 30 0.00000000 , -117.461205''')

In [7]:
t_reception_rcv = (2+1+9+6+7+6+1+3)*960
pseudorange_g06 = 22973727.891
pseudorange_g011 = 24719415.575
C = 299792458
wE =7.2921151467*(10**-5)

In [8]:
def emist(trec, pc, clk):
 
    df = pd.read_csv(clk, sep =",", names=['epoch','correction'])

    def convert_to_daysec(data,item):

        hour = float(data['epoch'][item].split(sep=' ')[0]) * 60 *60
        minute = float(data['epoch'][item].split(sep=' ')[1]) * 60 
        second = float(data['epoch'][item].split(sep=' ')[2])

        return hour + minute + second
    
    try: 
        for item in range(len(df)):
            df['epoch'][item] = convert_to_daysec(df,item)

    except : AttributeError

    def lagrange_interp(x, y, xi):

        n = len(x)
        w = np.zeros(n)
        yi = 0.0

        for i in range(n):
            w[i] = np.prod([(xi - x[j])/(x[i] - x[j]) for j in range(n) if j != i])
            yi += w[i]*y[i]

        return yi
    
    gps_lagrange = lagrange_interp(df['epoch'],df['correction'],trec)

    T_emission = trec - (pc / C) - (gps_lagrange*(10**-6)) # gps_lagrange is in microseconds !

    return(T_emission)
    print(T_emission)


In [13]:
emist_06 = emist(t_reception_rcv,pseudorange_g06,G06) 

In [14]:
emist_11 =emist(t_reception_rcv,pseudorange_g011,G11)

In [16]:
G06_t_x_y_z_correct = StringIO('''
10 45 0.00000000 , 2410.512326,22043.118635,14614.005543    ,577.445165
10 30 0.00000000 , 3299.994549,23294.509374,12297.196678  ,577.441900
10 15 0.00000000 , 3995.214624,24348.276586,9768.446681  ,577.438634
10 0 0.00000000 , 4515.588357,25166.028651,7071.147424  ,577.435389
9 45 0.00000000 , 4887.428596,25715.425240,4251.707854  ,577.432105
9 30 0.00000000 , 5142.736708, 25971.449758, 1358.755931  ,577.428825
9 15 0.00000000 , 5317.767894, 25917.413441, -1557.705986  ,577.425556
9 0 0.00000000 , 5451.426423,25545.649841,-4447.177167  ,577.422289
8 45 0.00000000 , 5583.554251,24857.870396,-7259.549756  ,577.419003
8 30 0.00000000 , 5753.181306,23865.165209,-9945.998220  ,577.415707
''')

G11_t_x_y_z_correct = StringIO('''
10 45 0.00000000 , 11917.794062 , 23687.004644 ,   1836.559085 , -117.548696
10 30 0.00000000 ,  12152.518761,  23615.400800 ,  -1025.035522,-117.538965
10 15 0.00000000 , 12258.490409 , 23261.665166 ,  -3869.022426 ,-117.529230
10 0 0.00000000 , 12272.055420  ,22615.762604  , -6646.542544, -117.519503
9 45 0.00000000 , 12231.496244  ,21676.324708  , -9309.861176, -117.509796
9 30 0.00000000 , 12175.388352  ,20450.818255  ,-11813.185945, -117.500047
9 15 0.00000000 , 12140.952623  ,18955.383870  ,-14113.452638, -117.490327
9 0 0.00000000 , 12162.467222 , 17214.350343, -16171.065529,  -117.480625
8 45 0.00000000 , 12269.800890 , 15259.442418, -17950.579469, -117.470932
8 30 0.00000000 , 12487.124925 , 13128.711717 ,-19421.312017  , -117.461205
''')

apporx_reciever_coords = [4239146.6414,  2886967.1245,  3778874.4800]


In [12]:
def sat_pos(trec, pc ,sp3, r_apr):
    
    df = pd.read_csv(sp3, sep =",", names=['epoch','X','Y','Z','correction'])

    def convert_to_daysec(data,item):

        hour = float(data['epoch'][item].split(sep=' ')[0]) * 60 *60
        minute = float(data['epoch'][item].split(sep=' ')[1]) * 60 
        second = float(data['epoch'][item].split(sep=' ')[2])

        return hour + minute + second

    try: 
        for item in range(len(df)):
            df['epoch'][item] = convert_to_daysec(df,item)

    except : AttributeError

    def lagrange_interp(x, y, xi):

        n = len(x)
        w = np.zeros(n)
        yi = 0.0

        for i in range(n):
            w[i] = np.prod([(xi - x[j])/(x[i] - x[j]) for j in range(n) if j != i])
            yi += w[i]*y[i]

        return yi

    X_LG = lagrange_interp(df['epoch'],df['X'],trec)
    Y_LG = lagrange_interp(df['epoch'],df['Y'],trec)
    Z_LG = lagrange_interp(df['epoch'],df['Z'],trec)

    dT_precise = math.sqrt((X_LG-r_apr[0])**2 + (Y_LG-r_apr[1])**2 + (Z_LG-r_apr[2])**2)/C

    def determine_rotation_matrices(angle , axis):
        
        if axis == 1:
            rotation_matrices = np.array([[1 , 0 , 0], [0 , math.cos(angle) ,math.sin(angle)],[0 , -math.sin(angle),math.cos(angle)]])

            return rotation_matrices

        elif axis == 2:
            rotation_matrices = np.array([[math.cos(angle),0,-math.sin(angle)], [0,1,0], [math.sin(angle),0, math.cos(angle)]])

            return rotation_matrices

        elif axis == 3:
            rotation_matrices = np.array([[math.cos(angle),math.sin(angle),0], [-math.sin(angle),math.cos(angle),0], [0,0,1]])

            return rotation_matrices

    def rotation(vector,angle,axis):
        
        if vector.shape == (3,) :
            vector = np.transpose(vector)
            rotation_matrix = determine_rotation_matrices(angle,axis)

            return (np.dot(vector,rotation_matrix))

        else:
            rotation_matrix = determine_rotation_matrices(angle,axis)

            return (np.dot(vector,rotation_matrix))
        
    ecef_cords = rotation(np.array((X_LG,Y_LG,Z_LG)),(wE * dT_precise),3)

    print(ecef_cords)


In [15]:
sat_pos(emist_06,pseudorange_g06,G06_t_x_y_z_correct,apporx_reciever_coords)

[ 5265.95215112 25970.54010524  -586.30504194]


In [17]:
sat_pos(emist_11,pseudorange_g011,G11_t_x_y_z_correct,apporx_reciever_coords)

[ 12147.79716274  19482.39102055 -13371.98981999]
